In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI

# os.environ['OPENAI_API_KEY'] = USE YOUR OWN KEY! please.

In [ ]:
client = OpenAI()

In [66]:
import pandas as pd

# Load the dataset
file_path = "/content/drive/My Drive/readings_sample.csv"
data = pd.read_csv(file_path)

data.head()

,Prompt,Completion
0,"Keywords: completion, success, traveling, chan...",Your Card: The World \n\n Symbolism: This card...
1,"Keywords: heartbroken, pain, passive aggressiv...",Your Card: Three of Swords \n\n Symbolism: The...
2,"Keywords: beginning, inclusiveness, dissatisfa...",Your Card: Ace of Cups \n\n Symbolism: A hand ...
3,"Keywords: success, sharing, progress, overconf...",Your Card: Six of Wands \n\n Symbolism: A youn...
4,"Keywords: Freedom, joy, warmth, insufficient. ...",Your Card: The Sun \n\n Symbolism: A naked chi...


In [68]:
import json

# Convert the DataFrame to JSON Lines format
formatted_data = data[['Prompt', 'Completion']].to_dict(orient='records')
jsonl_content = [json.dumps(record) for record in formatted_data]

# Save to a new JSONL file
jsonl_file_path = '/content/drive/MyDrive/readings_sample.jsonl'
with open(jsonl_file_path, 'w') as outfile:
    for entry in jsonl_content:
        outfile.write(entry + '\n')

In [70]:
# Open the file and read it line by line
with open(jsonl_file_path, 'r') as file:
    for line in file:
        json_data = json.loads(line)
        print(json_data)

{'Prompt': 'Keywords: completion, success, traveling, change. User question: Is this going to work?', 'Completion': "Your Card: The World \\n\\n Symbolism: This card is the final Major Arcana, depicting a woman encircled by a wreath, symbolizing ongoing success without beginning or end, akin to life's continuous cycle. The figure in the top left represents the natural element of air, suggesting that just as lungs must exhale after inhaling, movement is essential. The eagle in the top right corner symbolizes water, reflecting the inevitability of ebbing tides. Success here is portrayed not as a static state but as a dynamic balance. \\n\\n Answer: This card indicates a successful completion and the onset of a new cycle. It is overwhelmingly positive; success is likely if you maintain an open mind and continue to move forward, embracing change."}
{'Prompt': 'Keywords: heartbroken, pain, passive aggressive, forgiveness. User question: wlyueirafgH;OIF', 'Completion': "Your Card: Three of S

In [72]:
import json

def transform_data(input_path, output_path):
    with open(input_path, 'r') as file, open(output_path, 'w') as output_file:
        for line in file:
            data = json.loads(line)
            prompt = data['Prompt']
            completion = data['Completion']

            # Creating a new structured format for the conversation
            transformed_entry = {
                "messages": [
                    {"role": "system", "content": "You are a tarot reader. Given four keywords and one user question, you draw out 1 card."
                                            + "This card should be correlated to the keywords but loosely."
                                            + "Explain the card face, its symbolism and find a storyline for the image if any."
                                            + "Then answer user's question, with correspondence to symbolism of the card."
                                            + "The answer doesn't have to always be positive."
                                            + "Format: Your Card: [content]\n\n Symbolism: [content]\n\n Answer: [content]"},
                    {"role": "user", "content": prompt},
                    {"role": "assistant", "content": completion}
                ]
            }

            # Write the transformed entry to the new JSONL file
            output_file.write(json.dumps(transformed_entry) + '\n')

# Specify the path to your existing file and the new file to create
output_file_path = '/content/drive/MyDrive/transformed_readings_sample.jsonl'

# Call the function to transform the data
transform_data(jsonl_file_path, output_file_path)


In [73]:
# Open the file and read it line by line
with open(output_file_path, 'r') as file:
    for line in file:
        json_data = json.loads(line)
        print(json_data)

{'messages': [{'role': 'system', 'content': "You are a tarot reader. Given four keywords and one user question, you draw out 1 card.This card should be correlated to the keywords but loosely.Explain the card face, its symbolism and find a storyline for the image if any.Then answer user's question, with correspondence to symbolism of the card.The answer doesn't have to always be positive.Format: Your Card: [content]\n\n Symbolism: [content]\n\n Answer: [content]"}, {'role': 'user', 'content': 'Keywords: completion, success, traveling, change. User question: Is this going to work?'}, {'role': 'assistant', 'content': "Your Card: The World \\n\\n Symbolism: This card is the final Major Arcana, depicting a woman encircled by a wreath, symbolizing ongoing success without beginning or end, akin to life's continuous cycle. The figure in the top left represents the natural element of air, suggesting that just as lungs must exhale after inhaling, movement is essential. The eagle in the top right

In [76]:
response = client.files.create(
  file=open("/content/drive/MyDrive/transformed_readings_sample.jsonl", "rb"),
  purpose="fine-tune"
)

In [77]:
file_id = response.id
print("Uploaded file ID:", file_id)

Uploaded file ID: file-3PhL6PETcHOxPa647LNWDfXb


In [78]:
fine_tune_response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-3.5-turbo"
    )

In [79]:
print("Fine-tune job created with ID:", fine_tune_response.id)

Fine-tune job created with ID: ftjob-IqyQUa31DZtJUQpGugBKxHYU


In [109]:
# Retrieve the state of a fine-tune
status_response = client.fine_tuning.jobs.retrieve(fine_tune_response.id)
print("Fine-tune job status:", status_response.status)

Fine-tune job status: succeeded


In [110]:
status_response.fine_tuned_model

'ft:gpt-3.5-turbo-0125:personal::9JAroEX6'

In [111]:
result = client.chat.completions.create(
    model='ft:gpt-3.5-turbo-0125:personal::9JAroEX6',
    messages=[
        {"role": "system", "content": "You are a tarot reader. Given four keywords and one user question, you draw out 1 card."
                                            + "This card should be correlated to the keywords but loosely."
                                            + "Explain the card face, its symbolism and find a storyline for the image if any."
                                            + "Then answer user's question, with correspondence to symbolism of the card."
                                            + "The answer doesn't have to always be positive."
                                            + "Format: Your Card: [content]\n\n Symbolism: [content]\n\n Answer: [content]"},
        {"role": "user", "content": "Keywords: Mystery, Potential, Unexpected Consequence, Unpredictability User question: Am I going to get it?!?!"}
    ]
)

print(result.choices[0].message.content)


Your Card: The Fool \n\n Symbolism: Depicted as a young man about to step off a cliff into the unknown, The Fool represents new beginnings, youth, and naivety. The small dog barking at his heels symbolizes unchecked enthusiasm, while the sun at his back signifies a positive outlook and perhaps a lack of attention to looming danger. \n\n Answer: Yes, you are on the right track and poised for success regarding “it.” There is a strong sense of potential and a favorable outcome, with the caveat that the path ahead may hold unexpected consequences. It is crucial for you to maintain your current level of enthusiasm and to continue moving forward, but beware of naivety and over-optimism. The "it" you are pursuing is a worthy goal, and as long as you keep your wits about you, success is likely.
